In [2]:
!pip install tensorflow streamlit pyngrok Pillow

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
import os

In [4]:
pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os

base_dir = os.path.expanduser("C:/Users/adhar/Downloads/eyenova/kayser_fleischer")
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def preprocess_input(image):
    try:
        return tf.image.resize(image, (224, 224)) / 255.0
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode= 'nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)


# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 327 images belonging to 2 classes.
Found 70 images belonging to 2 classes.
Found 70 images belonging to 2 classes.


In [7]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [8]:
print(train_generator.samples)
print(val_generator.samples)
print(test_generator.samples)

steps_per_epoch = train_generator.samples // train_generator.batch_size

validation_steps = val_generator.samples // val_generator.batch_size

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    class_weight={1:31.5, 0:1}
)

327
70
70


c:\Users\adhar\miniconda3\envs\mlenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 33s 3s/step - accuracy: 0.6056 - loss: 13.2480 - val_accuracy: 0.7500 - val_loss: 2.0938
Epoch 2/10
 1/10 ━━━━━━━━━━━━━━━━━━━━ 4s 502ms/step - accuracy: 0.7143 - loss: 2.4920

c:\Users\adhar\miniconda3\envs\mlenv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 441ms/step - accuracy: 0.7143 - loss: 2.4920 - val_accuracy: 0.7500 - val_loss: 2.1652
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.7466 - loss: 2.3456 - val_accuracy: 0.7812 - val_loss: 2.1194
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 351ms/step - accuracy: 0.7188 - loss: 2.8204 - val_accuracy: 0.7812 - val_loss: 2.1206
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.7691 - loss: 2.3287 - val_accuracy: 0.7812 - val_loss: 2.0209
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 435ms/step - accuracy: 0.7143 - loss: 2.0985 - val_accuracy: 0.7500 - val_loss: 2.2893
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.7279 - loss: 2.4194 - val_accuracy: 0.7969 - val_loss: 1.6566
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 352ms/step - accuracy: 0.8438 - loss: 1.2738 - val_accuracy: 0.7656 - val_loss: 1.8880
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.7597 - loss: 1.8786 - val_accuracy: 0.7656 - val_loss: 1.

In [9]:
model.save('kfr_model.h5')

In [10]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 877ms/step - accuracy: 0.7646 - loss: 1.6186
Test Accuracy: 77.14%


In [11]:
#GLAUCOMA CLASSIFICATION

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

glaucoma_base_dir = 'C:/Users/adhar/Downloads/eyenova/glaucoma'
glaucoma_train_dir = os.path.join(glaucoma_base_dir, "train")
glaucoma_val_dir = os.path.join(glaucoma_base_dir, "val")
glaucoma_test_dir = os.path.join(glaucoma_base_dir, "test")

glaucoma_train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

glaucoma_val_datagen = ImageDataGenerator(rescale=1./255)
glaucoma_test_datagen = ImageDataGenerator(rescale=1./255)
glaucoma_train_generator = glaucoma_train_datagen.flow_from_directory(
    glaucoma_train_dir, target_size=(224, 224), batch_size=32, class_mode='binary'
)
glaucoma_val_generator = glaucoma_val_datagen.flow_from_directory(
    glaucoma_val_dir, target_size=(224, 224), batch_size=32, class_mode='binary'
)
glaucoma_test_generator = glaucoma_test_datagen.flow_from_directory(
    glaucoma_test_dir, target_size=(224, 224), batch_size=32, class_mode='binary',
    shuffle=False
)

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

glaucoma_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


glaucoma_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

glaucoma_model.summary()

Found 730 images belonging to 2 classes.
Found 165 images belonging to 2 classes.
Found 163 images belonging to 2 classes.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [12]:
history = glaucoma_model.fit(
    glaucoma_train_generator,
    validation_data=glaucoma_val_generator,
    epochs=10,
    steps_per_epoch=glaucoma_train_generator.samples // 32,
    validation_steps=glaucoma_val_generator.samples // 32
)

Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.5599 - loss: 0.7578 - val_accuracy: 0.6938 - val_loss: 0.6373
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 452ms/step - accuracy: 0.6250 - loss: 0.7014 - val_accuracy: 0.6875 - val_loss: 0.6425
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.6130 - loss: 0.7178 - val_accuracy: 0.6938 - val_loss: 0.6658
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 443ms/step - accuracy: 0.5312 - loss: 0.7305 - val_accuracy: 0.7063 - val_loss: 0.6609
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.6102 - loss: 0.6838 - val_accuracy: 0.6938 - val_loss: 0.6224
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 441ms/step - accuracy: 0.6875 - loss: 0.6030 - val_accuracy: 0.6875 - val_loss: 0.6187
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.6351 - loss: 0.6451 - val_accuracy: 0.6812 - val_loss: 0.6186
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 455ms/step - accuracy: 0.6875 - loss: 0.6057 - val_accuracy: 0.6938

In [ ]:
glaucoma_model.save('glaucoma_model.h5')

In [14]:
loss, accuracy = model.evaluate(glaucoma_test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.1576 - loss: 6.0468 
Test Accuracy: 32.52%


In [3]:
import os
print(os.getcwd())

c:\Users\adhar\Downloads\eyenovacode
